In [ ]:
import healpy as hp
import numpy as np
from scipy import optimize
import pandas as pd
import time

Read 100 modulated maps in NESST ordering, in this case flux threshold 1 $\mu$Jy and sample *z>0.5*. Can be changed.

In [ ]:
modmaps=[]
for i in range(0,100):
    modmaps.append(hp.read_map('path/simul1/modmapswo0_5/modmap%s.fits'%i,nest=True))

Define functions of estimator and its gradient, estimation of all parameters at the same time. The estimator function returns the value of the estimator with the input parameters; the gradient function returns an array of the derivatives over each component

In [ ]:
def estimator(x,mapa,average):
    pixels=[]
    for p in range(len(mapa)):
        if mapa[p] !=0:
            cosp=np.dot(x[1:],hp.pix2vec(64,p,nest=True))
            r=(mapa[p]-average*(1+x[0]*cosp))**2/(average*(1+x[0]*cosp))
            pixels.append(r)
    return np.sum(pixels)

In [ ]:
def grad(x,mapa,average):
    av=[]
    xv=[]
    yv=[]
    zv=[]
    for p in range(len(mapa)):
        if mapa[p] != 0:
            vec=hp.pix2vec(64,p,nest=True)
            cosp=np.dot(vec,x[1:]) 
            a=average*cosp-(mapa[p]**2*cosp)/(average*(x[0]*cosp+1)**2)
            av.append(a)
            h=x[0]*vec[0]*average-x[0]*vec[0]*mapa[p]**2/(average*(x[0]*cosp+1)**2)
            xv.append(h)
            y=x[0]*vec[1]*average-x[0]*vec[1]*mapa[p]**2/(average*(x[0]*cosp+1)**2)
            yv.append(y)
            z=x[0]*vec[2]*average-x[0]*vec[2]*mapa[p]**2/(average*(x[0]*cosp+1)**2)
            zv.append(z)
    return np.array([np.sum(av),np.sum(xv),np.sum(yv),np.sum(zv)])

Define initial guesses for the methods to use, they are the amplitude and direction of the CMB dipole.

In [ ]:
ing_vec=[4.62e-3,-0.06935272, -0.66218166,  0.7461271]

Optimization with Newton's conjugate gradient. Takes the map and the mean of all pixels as arguments

In [ ]:
ncg=[]
counts=0
totime=0
for m in modmaps:
    tim=time.time()
    print(counts)
    mean=np.mean(m)
    ncg.append(optimize.minimize(estimator,ing_vec,jac=grad,args=(m,mean),method='Newton-CG'))
    counts+=1
    tim2=time.time()-tim
    print(tim2)
    totime+=tim2
print(totime)

Convert the results to arrays and write in files.

In [ ]:
amp_ncg=[]
dire_ncg=[]
lonlat_ncg=[]
for i in ncg:
    amp_ncg.append(i.x[0])
    dire_ncg.append(i.x[1:])
    lonlat_ncg.append(hp.vec2ang(i.x[1:],lonlat=True))
    
newfile=pd.DataFrame(amp_ncg)
newfile.to_csv('path/simul1/wo0_5/amp.dat',sep=' ',index=None,header=None)
newfile=pd.DataFrame(dire_ncg)
newfile.to_csv('path/simul1/wo0_5/dir_vec.dat',sep=' ',index=None,header=None)
newfile=pd.DataFrame(lonlat_ncg)
newfile.to_csv('path/simul1/wo0_5/dir_lonlat.dat',sep=' ',index=None,header=None)


Furthermore, the estimators for other coordinates (pixel and spherical) are included below. The only thing needed to do to use them is to change the name of the estimator and the initial guess when calling the method.

In [ ]:
def estim_lonlat(x,mapa,average):
    pixels=[]
    for p in range(len(mapa)):
        cosp=np.dot(hp.ang2vec(x[1],x[2],lonlat=True),hp.pix2vec(64,p,nest=True))
        e=(mapa[p]-average*(1+x[0]*cosp))**2/(average*(1+x[0]*cosp))
        pixels.append(e)
    return np.sum(pixels)

In [ ]:
def grad_lonlat(x,mapa,average):
    av=[]
    lv=[]
    bv=[]
    amp,lon,lat=x
    for p in range(len(mapa)):
        if mapa[p] != 0:
            vec=hp.pix2vec(64,p,nest=True)
            h,y,z=vec
            cosp=np.dot(vec,hp.ang2vec(lon,lat,lonlat=True))
            a=average*cosp-(mapa[p]**2*cosp)/(average*(amp*cosp+1)**2)
            av.append(a)
            l=-((amp*np.cos(lat)*(y*np.cos(lon)-h*np.sin(lon))*(-amp*y*average*np.sin(lon)*np.cos(lat)-amp*h*average*np.cos(lon)*np.cos(lat)*-amp*average*z*np.sin(lat)-average+mapa[p])*(amp*y*average*np.sin(lon)*np.cos(lat)+amp*h*average*np.cos(lon)*np.cos(lat)+amp*average*z*np.sin(lat)+average+mapa[p]))/(average*(amp*y*np.sin(lon)*np.cos(lat)+amp*h*np.cos(lon)*np.cos(lat)+amp*z*np.sin(lat)+1)**2))
            lv.append(l)
            b=(amp*(np.sin(lat)*(y*np.sin(lon)+h*np.cos(lon))-z*np.cos(lat))*(-amp*average*np.cos(lat)*(y*np.sin(lon)+h*np.cos(lon))-amp*average*z*np.sin(lat)-average+mapa[p])*(amp*average*np.cos(lat)*(y*np.sin(lon)+h*np.cos(lon))+amp*average*z*np.sin(lat)+average+mapa[p]))/(average*(amp*np.cos(lat)*(y*np.sin(lon)+h*np.cos(lon))+amp*z*np.sin(lat)+1)**2)
            bv.append(b)
    return np.array([np.sum(av),np.sum(lv),np.sum(bv)])

In [ ]:
def estim_pix(x,mapa,average):
    pixels=[]
    for p in range(len(mapa)):
        if mapa[p] !=0:
            cosp=np.dot(x[1:],hp.pix2vec(64,p,nest=True))
            e=(mapa[p]-average*(1+x[0]*cosp))**2/(average*(1+x[0]*cosp))
            pixels.append(e)
    return np.sum(pixels)

Their respective initial guesses are the following, for the CMB dipole:

In [ ]:
ing_lonlat=[4.62e-3,264.021,48.253]
ing_pix=[4.62e-3,hp.ang2pix(32,264.021,48.253,nest=True,lonlat=True)]

Finally, to check whether the optimization has been done correctly the success atribute can be seen. The next loop stores the indexes of the maps with an incomplete optimization.

In [ ]:
succ=[]
for i in range(len(ncg)):
    if ncg[i].success==False:
        succ.append(i)